In [1]:
import torch
import math
from dataclasses import dataclass

from torch.autograd import Variable

In [2]:
# TODO - testme!!!!!!!!!!
# TODO - test identical results between c++ and python implementations

@dataclass
class BpParams:
    scale: Variable


class IncrementalBookFair:
        
    def __init__(self, params: BpParams, ref_price: float):
        self.params = params
        self.floating_ref_price = ref_price
        self.const_ref_price = ref_price
        
        self.bid_pressure = Variable(torch.zeros(1))
        self.ask_pressure = Variable(torch.zeros(1))
        
    def compute_fair_from_bid_ask(
        self
    ) -> torch.tensor:
        # For torch is it better to subtract logs or divide?
        # feel better to subtract logs
        ratio = torch.log(self.bid_pressure) - torch.log(self.ask_pressure)

        adjusted_fair = ratio / (2 * self.params.scale)
        
        return (adjusted_fair * self.const_ref_price) + self.floating_ref_price
    
    def add_impulse(self, price: float, impulse: float, bid: bool):
        ref_distance = (price - self.floating_ref_price) / self.const_ref_price
        
        if bid:
            distance = -ref_distance
        else:
            distance = ref_distance
        
        cost_of_impulse = torch.exp(-distance * self.params.scale)
        impulse_diff = cost_of_impulse * impulse
        if bid:
            self.bid_pressure += impulse_diff
        else:
            self.ask_pressure += impulse_diff
    
    
    def update_ref_price(self, new_ref_price: float):
        update_factor = math.exp(new_ref_price - self.floating_ref_price)
        
        # Bid pressure is sum of e^(P - P_ref)
        # Ask pressure is sum of e^(P_ref - P)
        # Bid transforms with multiplying by e^(p_ref_old - p_ref_new)
        # Ask transforms with multiplying by e^(p_ref_new - p_ref_old)
        # Intuitively, as the reference price gets lower, the cost of a given bid gets higher
        # and opposite for asks
        
        self.ask_pressure *= update_factor
        self.bid_pressure /= update_factor
        self.floating_ref_price = new_ref_price

In [3]:
incremental = IncrementalBookFair(BpParams(scale=Variable(torch.tensor(20000.0), requires_grad=True)), ref_price=10000)

In [4]:
incremental.add_impulse(price=10001, impulse=1, bid=False)

In [5]:
incremental.ask_pressure

tensor([0.1353], grad_fn=<AddBackward0>)

In [6]:
incremental.add_impulse(price=9999, impulse=2, bid=True)

In [7]:
incremental.add_impulse(price=9999, impulse=2, bid=True)

In [8]:
incremental.bid_pressure

tensor([0.5413], grad_fn=<AddBackward0>)

In [9]:
fair = incremental.compute_fair_from_bid_ask()

In [10]:
fair.backward()

In [11]:
incremental.params.scale.grad

tensor(-1.7329e-05)

In [12]:
fair

tensor([10000.3467], grad_fn=<AddBackward0>)

In [13]:
%load_ext tensorboard


In [14]:
from torchviz import make_dot

make_dot(fair).render("rnn_torchviz", format="png")

'rnn_torchviz.png'